# Enrichment with Scimago journal data

In [ ]:
import pandas as pd
import numpy as np
import textdistance as tx
from itertools import repeat
import time
import matplotlib.pyplot as plt
import statistics as st
from fuzzywuzzy import fuzz
%matplotlib inline

In [ ]:
punc = '''!()-[]{};:'"\,<>./?@#$%^*_~'''
punc2 = '&'
def limpieza(x):
    y=x
    for ele in y:
        if ele in punc:
            y = y.replace(ele, "")
        elif ele in punc2:
            y = y.replace('&','and')
    y=y.lower()
    y=y.strip()
    y=y.replace('  ',' ')
    y=y.replace('  ',' ')
    return(y)

In [ ]:
#path to the unified wos data
path = 'D:/trabajo CIAT/yuca/mis_datos/unificandodeag/datav6.csv'

data=pd.read_csv(path,low_memory=False)
data=np.unique(np.array(data['SO']).astype(str))
data=np.array(list(map(limpieza,data)))
tabla = pd.DataFrame(columns=['list','match','sim','year'])
tablat = pd.DataFrame(columns=['list','match','sim','year'])


#path to the folder containing all scimago files
path='D:/trabajo CIAT/yuca/mis_datos/varios/scimago/'

#first year to search on
first=1999

#last year to search on
last=2021

for i in range(first,last):
    start=time.time()

    crudo=pd.read_csv(path +'scimagojr ' + str(i) + '.csv',low_memory=False,sep=';')
    crudo=crudo['Title']
    crudo=np.array(list(map(limpieza,crudo)))
    tuples_list = [max([(fuzz.ratio(k,j),j) for j in crudo]) for k in data]
    similarity_score, fuzzy_match = map(list,zip(*tuples_list))
    
    # Create pandas DataFrame
    df = pd.DataFrame({"list":data, "match": fuzzy_match, "sim":similarity_score})
    df['year'] = list(repeat(i,df.shape[0]))
    dfa = df[df['sim']==100]
    dfa=dfa.reset_index(drop=True)
    dfb = df[df['sim']!=100]
    dfb=dfb.reset_index(drop=True)
    data = np.array(list(map(limpieza,list(dfb['list']))))
    tabla=tabla.append(dfa,ignore_index=True)
    if tablat.shape[0]<2:
        tablat=dfb
        tablat=tablat.reset_index(drop=True)
        print(i)
        print(time.time()-start)
        continue
    for j in range(dfb.shape[0]):
        indice=list(tablat['list']).index(list(dfb['list'])[j])
        if list(dfb['sim'])[j] > list(tablat['sim'])[indice]:
            tablat.loc[indice,'sim']=dfb.loc[j,'sim']
            tablat.loc[indice,'match']=dfb.loc[j,'match']
            tablat.loc[indice,'year']=i
    boleano=[]
    for x in list(tablat['list']):
        if (x in list(dfa['list'])):
            boleano.append(False)
        else:
            boleano.append(True)
    tablat=tablat[boleano]
    tablat=tablat.reset_index(drop=True)
    print(i)
    print(time.time()-start)

In [ ]:
print(tablat.shape)
print(tabla.shape)
print(len(data))
print(tablat.shape[0]+tabla.shape[0])

In [ ]:
#A table 'tablat.csv' will be written, were non-perfect journal name matches will be shown, 
#it is necesary to manually add a column called 'confirm' and fill in a '1' (without quotation marks) 
#to indicate a succesfull match

tabla.to_csv('tabla.csv')
tablat.to_csv('tablat.csv')
dfa.to_csv('dfa.csv')
dfb.to_csv('dfb.csv')

In [ ]:
tabla=pd.read_csv('tabla.csv')
tablat=pd.read_csv('tablat.csv')

tablat=tablat[tablat['confirm']==1]

vlist=list(tablat.list)
mlist=list(tablat.match)
slist=list(tablat.sim)
ylist=list(tablat.year)

datat=pd.DataFrame(list(zip(vlist,mlist,slist,ylist)),columns=['list', 'match', 'sim', 'year'])

mapa=datat.append(tabla,ignore_index=True)

mapa

In [ ]:
punc = '''!()-[]{};:'"\,<>./?@#$%^*_~'''
punc2 = '&'
def limpieza(x):
    y=x
    for ele in y:
        if ele in punc:
            y = y.replace(ele, "")
        elif ele in punc2:
            y = y.replace('&','and')
    y=y.lower()
    y=y.strip()
    y=y.replace('  ',' ')
    y=y.replace('  ',' ')
    return(y)

In [ ]:
#path to the latest year of scimago data 
path='D:/trabajo CIAT/yuca/mis_datos/varios/scimago/scimagojr 2020.csv'
crudo=pd.read_csv(path,low_memory=False,sep=';')

#path to the unified wos data
path = 'D:/trabajo CIAT/yuca/mis_datos/unificandodeag/datav6.csv'
data=pd.read_csv(path,low_memory=False)

In [ ]:
titulos=list(map(limpieza,(list(crudo.Title))))
ini=list(map(limpieza,(list(data['SO'].astype(str)))))

In [ ]:
#path to the folder containing all scimago files
path='D:/trabajo CIAT/yuca/mis_datos/varios/scimago/'

#first year to search on
first=1999

#last year to search on
last=2021


truemap=pd.DataFrame()
for j in range(first,last):
    crudo=pd.read_csv(path + 'scimagojr ' + str(j) + '.csv',low_memory=False,sep=';')
    titulos=list(map(limpieza,(list(crudo.Title))))
    indices=[]
    for i in ini:
        try:
            indices.append(titulos.index(i))
        except:
            indices.append(-1)
    truemap[str(j)] = indices
    #truemap=truemap.assign(j = indices)
    #truemap=truemap.insert(indices)

In [ ]:
max(indices)

In [ ]:
len(np.unique(np.array(indices)))

In [ ]:
truemap[0:10]

In [ ]:
mapa

In [ ]:
def encontar(x):
    if(max(x)!=-1):
        return(1)
    else:
        return(0)

In [ ]:
test=truemap.apply(encontar,axis=1)
#test=np.array(list(test))

## Perfect find percentage

In [ ]:
st.mean(test)

## Finding year and indexing

In [ ]:
# x is the row of values of dataframe data(['SO'])
# returns the closest year where it was found, and the index of its specific 'raw'

def encontrar(x,ano):
    #first year to search on
    first=1999

    #last year to search on
    last=2020
    
    encontrado=True
    bias=0
    
    if(ano<first):
        ano=first
    if(ano>last):
        ano=last
    if(max(list(truemap.iloc[x,:]))==-1):
        return (-1,-1)
    while encontrado:
        if ano + bias <= last:
            numero=truemap[str(ano+bias)][x]
            if numero!= -1:
                return(numero,ano+bias)
        if ano - bias >= first:
            if numero!= -1:
                return(numero,ano-bias)
        bias+=1
        if ano + bias > last and ano - bias < first:
            return(-2,-2)

In [ ]:
anos=list(data['PY'].replace(np.nan, last).astype(int))

In [ ]:
indices=list(map(encontrar,range(data.shape[0]),anos))

In [ ]:
indicesdef = list(list(zip(*indices))[0])
anosdef = list(list(zip(*indices))[1])

In [ ]:
#first year to search on
first=1999

#last year to search on
last=2021

dataframes=list()
for j in range(first, last):
    #path to the folder containing all scimago files
    path='D:/trabajo CIAT/yuca/mis_datos/varios/scimago/'
    crudo=pd.read_csv(path + 'scimagojr ' + str(j) + '.csv',low_memory=False,sep=';')
    titulos=list(map(limpieza,(list(crudo.Title))))
    crudo['Title']=titulos
    dataframes.append(crudo)

In [ ]:
lista=[]
for i in range(data.shape[0]):
    if(anosdef[i])<0:
        lista.append(list(np.repeat(np.nan,20)))
    else:
        lista.append(list(dataframes[anosdef[i]-1999].iloc[indicesdef[i],:]))

In [ ]:
datafinal=pd.DataFrame(lista,columns=crudo.columns)

In [ ]:
datafinal

In [ ]:
#path to the unified wos data
path = 'D:/trabajo CIAT/yuca/mis_datos/unificandodeag/datav8.csv'

data2=pd.read_csv(path,low_memory=False)

In [ ]:
data2=data2.iloc[:,5:]
for i in range(len(list(crudo.columns))):
    if(i!=2):
        data2[list(crudo.columns)[i]]=datafinal[list(crudo.columns)[i]]
    else:
        data2['Journal']=datafinal[list(crudo.columns)[i]]

In [ ]:
#Path to the new unified file
path = 'D:/trabajo CIAT/yuca/mis_datos/unificandodeag/datav9.xlsx'
data2.to_excel(path,index=False)